In [ ]:
# Add 'hide code' button
from IPython.display import HTML
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
Der Code für dieses IPython-Notebook ist zur leichteren Lesbarkeit standardmäßig ausgeblendet. Um den Code ein-/auszuschalten, klicken Sie auf <a href="javascript:code_toggle()">hier</a>.''')

# Klimawandel

Das Klima ist im Wandel. Die Erderwärmung ist längst kein Mythos mehr. Doch wie wird der Klimawandel gemessen? Wie sieht die Temperaturveränderung und das Treibhausgas CO<sub>2</sub> der einzelnen Länder aus? Wie verhält sich der CO<sub>2</sub> Ausstoss eine Landes gegenüber dem Globalen Ausstoss?

Mit den folgenen Darstellungen versuchen wir, diese Fragen zu beantworten. Als erstes sehen Sie die globale jährliche Temperaturveränderung von 1900 bis 2013. Der nächste Plot zeigt die monatlich gemessene Temperatur der einzelnen Länder von 1900 bis 2013, begleitet mit einer gegläteten Temperaturkurve. Es folgt eine Darstellung der jährlichen CO<sub>2</sub>-Emissionen der einzelnen Länder von 1960 bis 2018. Zu guter Letzt folgt eine Darstellung der jährlichen globalen CO<sub>2</sub>-Emissionen, um die Kurven miteinander zu vergleichen. Das CO<sub>2</sub> wird in MtCO<sub>2</sub> ausgewiesen.

Die Erwärmung eines Landes oder gar des Globus zu zeigen ist nicht trivial. Die einzelnen Messstationen haben meist nicht die gleiche Höhe, sind geographisch unterschiedlich verteilt, sind der Witterung verschieden ausgesetzt und die Messungen können verzerrt sein. Eine Verrechnung dieser Werte ist sehr schwierig. Es scheint gar unmöglich zu sagen, dass ein Land eine Durchschnittstemperatur während einer Periode hat. Doch wie kann der Klimawandel trotzdem gemessen werden? Eine Mögliche Antwort dazu ist die Temperatur mit Hilfe bestimmter Verfahren wie z.B. der Berkeley Earth abzuschätzen.
<br><br>

**Berkeley Earth Temperatur**

Die Länder enthalten eine extrahierte regionale Zusammenfassung der Ergebnisse der Landoberflächentemperatur, die mit der Mittelwertmethode von Berkeley Earth für die Region ermittelt wurden.
Die Berkeley-Earth-Methode nimmt Temperaturbeobachtungen von einer großen Sammlung von Wetterbeobachtungsstationen und erstellt eine Schätzung des zugrunde liegenden globalen Temperaturfeldes über alle Landgebiete der Erde.  Sobald dieses Temperaturfeld erzeugt wurde, ist es möglich, die Temperaturentwicklung einzelner Regionen einfach durch Integration des Feldes über der betreffenden Region abzuschätzen. 
Die Temperaturen werden in Celsius angegeben und als Anomalien im Vergleich zum Durchschnitt der Monate Januar 1951 bis Dezember 1980 gemeldet.

**Plots**

Die Plots sind interaktiv bedienbar, um die Daten akkurat zeigen zu können. Jeder der Plots besitzt Standardinteraktivität (wie z.B. zoom, hover oder an- und abwählen der gezeigten Daten mittels Legende), mit welcher die Daten besser exploriert werden können.

Zusätzlich bietet der Plot der "Weltkarte Temperatur" eine Animationsmöglichkeit, um die globale Temperatur jährlich zu zeigen. 
Die weiteren Plots können mit den Dropdowns die entsprechenden Länder ausgewählt werden. Mmit Hilfe der Schiebefensters kann beliebig genau ein Zeitintervall gewählt werden, welches auch verschiebbar ist.

In [ ]:
import sys, json
sys.path.insert(0, '../')
from database.get_db_data import DataAccess
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
dba = DataAccess()
df_gd = dba.get_global_data_db()

df_gco2 = df_gd['co2'].sort_values(by = ['country', 'date'])

df_gtemp = df_gd['temperature'].sort_values(by = ['country', 'date'])
df_gtemp = df_gtemp[df_gtemp.date.dt.year >= 1880].reset_index(drop = True) # pre-filtering is necessary to prevent .rolling issues
df_gtemp['yearly_mean_temperature'] = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year])['temperature'].transform('mean')
df_gtemp['rolling_temperature_5y'] = df_gtemp.groupby(by = 'country')['temperature'].rolling(window = 60, min_periods = 48).mean().reset_index().loc[:, 'temperature'] # 60 months = 5 years, min rquired 4 years
df_gtemp['yearly_mean_temperature_rolling_5y'] = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year])['rolling_temperature_5y'].transform('mean')
df_gtemp = df_gtemp[df_gtemp.date.dt.year >= 1900].reset_index(drop = True) # post-filtering to have the .rolling and monthly data available for most countries
df_gtemp_yearly = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year]).first().drop(columns = ['date', 'temperature']).reset_index()

## Weltkarte Temperatur

Diese Animation zeigt die jährliche Berkeley Earth Temperaturveränderung von 1900 bis 2013. Die Temperaturkurven wurden geglättet, in dem in jedem Monat der Durchscnitt der Monatswerte der letzten 5 Jahre genommen wurde. Daraus werden järhliche Mittelwerte gebildet.

Dies erzeugt einen konstanteren Temperaturverlauf über die Jahre, wobei Ausreisserjahre nicht so stark ins Gewicht fallen.

In [ ]:
def plot_global_temp_choro(df_gtemp_yearly):
    yearly_min = df_gtemp_yearly.loc[:, 'yearly_mean_temperature_rolling_5y'].min()
    yearly_max = df_gtemp_yearly.loc[:, 'yearly_mean_temperature_rolling_5y'].max()

    fig = px.choropleth(data_frame = df_gtemp_yearly,
                        title = '<br>Globale Temperaturveränderung',
                        locations = 'alpha_3',
                        color = 'yearly_mean_temperature_rolling_5y',  # value in column 'Confirmed' determines color
                        hover_name = 'country',
                        color_continuous_scale = 'Jet',  #  color scale red, yellow green
                        range_color = [yearly_min, yearly_max],
                        animation_frame = 'date',
                        width = 1400, 
                        height = 1000)

    fig.update_layout(
        margin = dict(l = 70, r = 70, t = 20, b = 0, pad = 4),
        paper_bgcolor = 'LightSteelBlue',
        title_font_size = 50, 
        coloraxis_colorbar = dict(
            title = 'Berkeley Earth<br>Temperatur',
            thicknessmode = 'pixels', thickness = 40,
            lenmode = 'pixels', len = 620,
            ticks = 'outside', 
            ticksuffix = r' °C',
            dtick = 1),
        title = {
            "yref": "paper",
            "y" : 1,
            "yanchor" : "bottom"})

    fig.show()
    
plot_global_temp_choro(df_gtemp_yearly)

## Temperaturverlauf der Länder

Diese Darstellung zeigt die monatliche Berkely Earth Temperatur des ausgewählten Landes von 1900 bis 2013. Da die Monatswerte stark schwanken, ist auch ein Trend schwer erkennbar. Aus diesem Grund wurde noch eine geglättete Kurve hinzugefügt, bei welcher in jedem Monat der Durchscnitt der Monatswerte der letzten 5 Jahre genommen wurde. 

In [ ]:
def plot_selective_lineplot(df, measuring_unit):
    """
    Plots a linechart for either countrywise Berkeley Earth Temperature or CO_2
    
    :param df: pd.DataFrame, data to plot
    :param measuring_unit: str, either 'temp' to plot temperature or 'co2' to plot global co2
    """
    if measuring_unit == 'temp':
        x, y = 'date', 'temperature'
        color = '#DC3220'
        title_prefix = 'Berkeley Earth Temperatur'
        yaxis_title = 'Temperature °C'
    elif measuring_unit == 'co2':
        x, y = 'date', 'co2'
        color = '#000000'
        title_prefix = 'Jährliche CO<sub>2</sub> Emmissionen'
        yaxis_title = 'MtCO<sub>2</sub>'
    else:
        raise ValueError("Only 'temp' or 'co2' is a valid value for `measuring_unit`")

    fig = go.FigureWidget(layout = {'height': 600})
    countries = list(df.loc[:, 'country'].unique())
    buttons = []
    
    first, vis = True, True
    for i in range(len(countries)):
        country = countries[i]
        data = df[df.loc[:, 'country'] == country]
        fig.add_trace(
            go.Scatter(
                x = data.loc[:, x],
                y = data.loc[:, y],
                name = country,
                line = dict(color = color),
                visible = vis,
            ))
        if measuring_unit == 'temp':
            fig.add_trace(
                go.Scatter(
                    x = data.loc[:, x],
                    y = data.loc[:, 'rolling_temperature_5y'],
                    name = country + ' geglättet',
                    line = dict(color = color),
                    visible = vis,
                ))
        if first:
            first, vis = False, False
            fig.update_layout(title = title_prefix + ' von ' + country)
        
        if measuring_unit == 'temp':
            visible = [False] * 2 * len(countries)
            visible[2 * i] = True
            visible[2 * i + 1] = True
        else:
            visible = [False] * len(countries)
            visible[i] = True
        new_button = dict(label = country,
                          method = 'update',
                          args = [{'visible': visible},
                                # the index of True aligns with the indices of plot traces
                                {'title': title_prefix + ' von ' + country,
                                 'showlegend': True}])
        buttons.append(new_button)

    # Add dropdown
    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = buttons,
            direction = 'down',
            pad = {'r': 10, 't': 10},
            showactive = True,
            x = 1,
            xanchor = 'right',
            y = 1,
            yanchor = 'bottom')],
        xaxis_title = 'Zeit',
        yaxis_title = yaxis_title,
    )

    # Add range slider
    fig.update_layout(
        xaxis = dict(
            rangeselector = dict(
                buttons = list([
                    dict(count = 1,
                         label = '1 Jahr',
                         step = 'year',
                         stepmode = 'backward'),
                    dict(step = 'all',
                         label = 'alles')
                ])
            ),
            rangeslider = dict(visible = True),
            type = 'date'
        )
    )

    fig.show()


In [ ]:
plot_selective_lineplot(df_gtemp, 'temp')

## Emmissionen der Länder

Diese Darstellung zeigt die jährlichen CO<sub>2</sub>-Emmissionen des ausgewählten Landes von 1960 bis 2019 in MtCO<sub>2</sub>.

In [ ]:
plot_selective_lineplot(df_gco2, 'co2')

## Globale Emmissionen

Diese Darstellung zeit die jährlichen weltweiten CO<sub>2</sub>-Emmissionen von 1960 bis 2019 in MtCO<sub>2</sub>.

Diese wurde gebildet aus der Summe der Emmissionen der einzelnen Länder. Da es teilweise fehlende Werte in den Daten gibt, werden diese mit dem Jahresmittelwert der vorhandenen Jahre ausgeglichen, um den Fehler minimal zu halten.

In [ ]:
def plot_global_co2(df):
    """
    Plots the global CO_2 emmisions in a line chart
    
    :param df: pd.DataFrame, data to plot
    """
    fig = go.FigureWidget(layout = {'height': 600})
    
    fig.add_trace(
        go.Scatter(
            x = df.loc[:, 'date'],
            y = df.loc[:, 'co2'],
            name = 'Globale CO<sub>2</sub> Emmissionen',
            line = dict(color = '#000000')
        ))

    # Add range slider
    fig.update_layout(
        xaxis_title = 'Zeit',
        yaxis_title = 'MtCO<sub>2</sub>',
        title = 'Jährliche Globale CO<sub>2</sub> Emmissionen',
        xaxis = dict(
            rangeselector = dict(
                buttons = list([
                    dict(count = 1,
                         label = '1 Jahr',
                         step = 'year',
                         stepmode = 'backward'),
                    dict(step = 'all',
                         label = 'alles')
                ])
            ),
            rangeslider = dict(visible = True),
            type = 'date'
        )
    )

    fig.show()


# fill the missing 'co2' values of each country with countrywise co2.mean() to minimize global error
df_gco2_filled = df_gco2.copy()
df_gco2_filled['co2'] = df_gco2_filled.groupby('country').transform(lambda x: x.fillna(x.mean()))['co2']
df_global_co2 = df_gco2_filled.groupby(by = 'date').sum().reset_index()
plot_global_co2(df_global_co2)